In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


X = df[dataset.feature_names].values
y = dataset.target
predictions = np.zeros_like(y)

kfold = KFold(n_splits=5, shuffle=True)
for train_idx, valid_idx in kfold.split(X):
    X_train, X_valid = X[train_idx], X[valid_idx]
    y_train, y_valid = y[train_idx], y[valid_idx]

    preprocessor = StandardScaler().fit(X_train)
    X_train, X_valid = preprocessor.transform(X_train), preprocessor.transform(X_valid)

    # model = LogisticRegression()
    model = LinearDiscriminantAnalysis()
    model.fit(X_train, y_train)
    predictions[valid_idx] = model.predict(X_valid)

accuracy = (y == predictions).mean()
wrong_rows = np.where(y != predictions)[0]

print('Accuracy: {:0.2f}%'.format(100*accuracy))
wrong_df = df.loc[wrong_rows]
wrong_df

In [ ]:
def cartesian_product(arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

num_samples = 25
arrays = [np.linspace(X[:, i].min(), X[:, i].max(), num_samples) for i in range(len(dataset.feature_names))]
grid = cartesian_product(arrays)

grid_predictions0 = np.zeros((num_samples**4, 2))
grid_predictions1 = grid_predictions0.copy()

grid = (grid - X.mean(axis=0)) / X.std(axis=0)
X = (X - X.mean(axis=0)) / X.std(axis=0)

predictions = np.zeros_like(y)

kfold = KFold(n_splits=5, shuffle=True)
for train_idx, valid_idx in kfold.split(X):
    X_train, X_valid = X[train_idx], X[valid_idx]
    y_train, y_valid = y[train_idx], y[valid_idx]

#     preprocessor = StandardScaler().fit(X_train)
#     X_train, X_valid = preprocessor.transform(X_train), preprocessor.transform(X_valid)

#     model = LogisticRegression()
#     model.fit(X_train, y_train)
#     predictions[valid_idx] = model.predict(X_valid)
#     grid_predictions += model.predict_proba(grid)

    model0 = LinearDiscriminantAnalysis()
    model0.fit(X_train, (y_train>0).astype('int'))
    preds0 = model0.predict(X_valid)

    X_train = X_train[y_train > 0]
    y_train = y_train[y_train > 0]

    model1 = LinearDiscriminantAnalysis()
    model1.fit(X_train, y_train-1)
    preds1 = model1.predict(X_valid)

    # input_grid = preprocessor.transform(grid)
    grid_predictions0 += model0.predict_proba(grid)
    grid_predictions1 += model1.predict_proba(grid)

    preds = preds0
    preds[preds == 1] = preds1[preds == 1] + 1
    predictions[valid_idx] = preds

accuracy = (y == predictions).mean()
wrong_rows = np.where(y != predictions)[0]

print('Accuracy: {:0.2f}%'.format(100*accuracy))
wrong_df = df.loc[wrong_rows]
print(wrong_df)

# grid_predictions = grid_predictions.argmax(axis=1)

grid_predictions0 /= 5
grid_predictions1 /= 5

grid_predictions = grid_predictions0.argmax(axis=1)
grid_predictions[grid_predictions == 1] = grid_predictions1[grid_predictions==1].argmax(axis=1)+1

In [ ]:
# grid = (grid - X.mean(axis=0)) / X.std(axis=0)
# X = (X - X.mean(axis=0)) / X.std(axis=0)

eigvals, eigvecs = eig(X.T @ X)
W = eigvecs[np.argsort(eigvals)[:2]].real
X_2d = X @ W.T
grid_transformed = grid @ W.T

In [ ]:
from scipy.stats import mode

arrays = [np.linspace(X_2d[:, i].min(), X_2d[:, i].max(), num_samples) for i in range(2)]
grid_2d = cartesian_product(arrays)

k = 10
predictions = []
for sample in grid_2d:
    distances = ((sample - grid_transformed)**2).sum(axis=1)
    top_k = grid_predictions[np.argsort(distances)[-k:]]
    predictions.append(mode(top_k).mode[0])

image = np.array(predictions).reshape(num_samples, num_samples)
p.image(
    image=[image.T[::-1, ::-1]],
    x=arrays[0][0],
    y=arrays[1][0],
    dw=arrays[0][-1]-arrays[0][0],
    dh=arrays[1][-1]-arrays[1][0],
    palette=palette,
    global_alpha=0.25,
    level="image"
)
show(p)